# Toronto Neighborhood Clustering

### Introduction

In this notebook, I will use the Foursquare API to explore neighborhoods in Toronto. I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

Before importing the data, download the necessary libraries and dependencies.

In [1]:
import pandas as pd # library for data analysis

import numpy as np # handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means for clustering
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Explore the Dataset

In [2]:
# Import latitude and longitude data and clean it
latlon_df = pd.read_csv('Geospatial_Coordinates.csv')
latlon_df.rename({'Postal Code': 'PostalCode'}, axis=1, inplace=True)

# Import postal code and borough data
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Rename column and remove 'Not assigned' rows
df.rename({'Postal Code': 'PostalCode'}, axis=1, inplace=True)
df = df[df['Borough'] != 'Not assigned']

# Join datasets on postal code column
df_complete = pd.merge(left=df, right=latlon_df, how='left', left_on='PostalCode', right_on='PostalCode')
df_complete.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Now I will use the Geopy library to get the latitude and longitude values of Toronto.

In [3]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of Toronto using lat and lon values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Borough'], df_complete['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)
    
map_toronto

For simplicity, we will narrow down the map to only show boroughs with Toronto in the name. We will slice the original dataframe and create a new dataframe, toronto_data.

In [9]:
toronto_data = df_complete[df_complete['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Create a map of boroughs including Toronto in the name with neighborhoods superimposed on top.

In [13]:
# create map of Toronto using lat and lon values
map_toronto_2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto_2)
    
map_toronto_2

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'EE1M22TO3BHRKZ0GIZILLR3YDXX3OKSPYH1QSQAIDKSGTNVO' # your Foursquare ID
CLIENT_SECRET = 'TQ5P241BMDO53W2MT4J3OBK2RKFU0NPLOUB5MLIXXAM11TW3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EE1M22TO3BHRKZ0GIZILLR3YDXX3OKSPYH1QSQAIDKSGTNVO
CLIENT_SECRET:TQ5P241BMDO53W2MT4J3OBK2RKFU0NPLOUB5MLIXXAM11TW3


Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [18]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Let's get the neighborhoods lat and lon values.

In [20]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.
